Terminal

In [1]:
# %cd data
# !pwd
# !gunzip GoogleNews-vectors-negative300.bin
# %cd ..
# url_to_word_embedding_zip = 'https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz'
# import os.path
# from os import path
# url_with_extention_removed = os.path.splitext(url_to_word_embedding_zip)[0]
# unzipped_file_name = os.path.basename(url_with_extention_removed)
# desired_path_of_unzipped_word_embedding = "data/" + unzipped_file_name+'.gz'
# print(url_to_word_embedding_zip)
# !gunzip $url_to_word_embedding_zip $desired_path_of_unzipped_word_embedding

# !pwd
# !ls "data/"
# !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz" -P "data/"
# !ls
# !gunzip "data/GoogleNews-vectors-negative300.bin"
# !ls "data/"

!ls "data/"

ls: cannot access 'data/': No such file or directory


INPUTS

In [0]:
#repoPath should start with the path 
#'/content/gdrive/My Drive/' and go to the folder where the git repo is, and should end with the name of the repo:
repoPath = '/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance'

Mount google drive

In [6]:
from google.colab import drive
drive.mount('/content/gdrive')
import os
os.chdir(repoPath)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


Pip Install packages, and import general packages

In [4]:
#pip installs
! pip install ipdb -q
import ipdb 
#place the line of code below from whichever line you want to start debugging
#type n for step over and s for step into
# ipdb.set_trace()

from time import time
import gensim
import pandas as pd
from typing import List, Tuple
import numpy as np

In [0]:

from collections import OrderedDict
import numpy as np
import spacy
from spacy.lang.en.stop_words import STOP_WORDS

nlp = spacy.load('en_core_web_sm')

class TextRank4Keyword():
    """Extract keywords from text"""
    
    def __init__(self):
        self.d = 0.85 # damping coefficient, usually is .85
        self.min_diff = 1e-5 # convergence threshold
        self.steps = 10 # iteration steps
        self.node_weight = None # save keywords and its weight

    
    def set_stopwords(self, stopwords):  
        """Set stop words"""
        for word in STOP_WORDS.union(set(stopwords)):
            lexeme = nlp.vocab[word]
            lexeme.is_stop = True
    
    def sentence_segment(self, doc, candidate_pos, lower):
        """Store those words only in cadidate_pos"""
        sentences = []
        for sent in doc.sents:
            selected_words = []
            for token in sent:
                # Store words only with cadidate POS tag
                if token.pos_ in candidate_pos and token.is_stop is False:
                    if lower is True:
                        selected_words.append(token.text.lower())
                    else:
                        selected_words.append(token.text)
            sentences.append(selected_words)
        return sentences
        
    def get_vocab(self, sentences):
        """Get all tokens"""
        vocab = OrderedDict()
        i = 0
        for sentence in sentences:
            for word in sentence:
                if word not in vocab:
                    vocab[word] = i
                    i += 1
        return vocab
    
    def get_token_pairs(self, window_size, sentences):
        """Build token_pairs from windows in sentences"""
        token_pairs = list()
        for sentence in sentences:
            for i, word in enumerate(sentence):
                for j in range(i+1, i+window_size):
                    if j >= len(sentence):
                        break
                    pair = (word, sentence[j])
                    if pair not in token_pairs:
                        token_pairs.append(pair)
        return token_pairs
        
    def symmetrize(self, a):
        return a + a.T - np.diag(a.diagonal())
    
    def get_matrix(self, vocab, token_pairs):
        """Get normalized matrix"""
        # Build matrix
        vocab_size = len(vocab)
        g = np.zeros((vocab_size, vocab_size), dtype='float')
        for word1, word2 in token_pairs:
            i, j = vocab[word1], vocab[word2]
            g[i][j] = 1
            
        # Get Symmeric matrix
        g = self.symmetrize(g)
        
        # Normalize matrix by column
        norm = np.sum(g, axis=0)
        g_norm = np.divide(g, norm, where=norm!=0) # this is ignore the 0 element in norm
        
        return g_norm

    
    def get_keywords(self, number=10):
        """Print top number keywords"""
        node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
        for i, (key, value) in enumerate(node_weight.items()):
            print(key + ' - ' + str(value))
            if i > number:
                break

    def get_list_of_strings_of_the_top_textrank_keywords_in_order_of_textrank(self, number=10):
      list_of_top_keywords = []

      """Print top number keywords"""
      node_weight = OrderedDict(sorted(self.node_weight.items(), key=lambda t: t[1], reverse=True))
      for i, (key, value) in enumerate(node_weight.items()):
          #print(key + ' - ' + str(value))
          list_of_top_keywords.append(key.lower())
          if i > number:
            break
      return list_of_top_keywords       
        
    def analyze(self, text, 
                candidate_pos=['NOUN', 'PROPN'], 
                window_size=4, lower=False, stopwords=list()):
        """Main function to analyze text"""
        
        # Set stop words
        self.set_stopwords(stopwords)
        
        # Pare text by spaCy
        doc = nlp(text)
        
        # Filter sentences
        sentences = self.sentence_segment(doc, candidate_pos, lower) # list of list of words
        
        # Build vocabulary
        vocab = self.get_vocab(sentences)
        
        # Get token_pairs from windows
        token_pairs = self.get_token_pairs(window_size, sentences)
        
        # Get normalized matrix
        g = self.get_matrix(vocab, token_pairs)
        
        # Initionlization for weight(pagerank value)
        pr = np.array([1] * len(vocab))
        
        # Iteration
        previous_pr = 0
        for epoch in range(self.steps):
            pr = (1-self.d) + self.d * np.dot(g, pr)
            if abs(previous_pr - sum(pr))  < self.min_diff:
                break
            else:
                previous_pr = sum(pr)

        # Get weight for each node
        node_weight = dict()
        for word, index in vocab.items():
            node_weight[word] = pr[index]
        
        self.node_weight = node_weight

In [25]:
text = '''
Sony PlayStation 4 (PS4) (Latest Model)- 500 GB Jet Black Console The PlayStation 4 system opens the door to an incredible journey through immersive new gaming worlds and a deeply connected gaming community. Step into living, breathing worlds where you are hero of your epic journey. Explore gritty urban environments, vast galactic landscapes, and fantastic historical settings brought to life on an epic scale, without limits. With an astounding launch lineup and over 180 games in development the PS4 system offers more top-tier blockbusters and inventive indie hits than any other next-gen console. The PS4 system is developer inspired, gamer focused. The PS4 system learns how you play and intuitively curates the content you use most often. Fire it up, and your PS4 system points the way to new, amazing experiences you can jump into alone or with friends. Create your own legend using a sophisticated, intuitive network built for gamers. Broadcast your gameplay live and direct to the world, complete with your commentary. Or immortalize your most epic moments and share at the press of a button. Access the best in music, movies, sports and television. PS4 system doesn t require a membership fee to access your digital entertainment subscriptions. You get the full spectrum of entertainment that matters to you on the PS4 system. PlayStation 4: The Best Place to Play The PlayStation 4 system provides dynamic, connected gaming, powerful graphics and speed, intelligent personalization, deeply integrated social capabilities, and innovative second-screen features. Combining unparalleled content, immersive gaming experiences, all of your favorite digital entertainment apps, and PlayStation exclusives, the PS4 system focuses on the gamers.Gamer Focused, Developer InspiredThe PS4 system focuses on the gamer, ensuring that the very best games and the most immersive experiences are possible on the platform.<br>Read more about the PS4 on ebay guides.</br>
'''
tr4w = TextRank4Keyword()
tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN'], window_size=4, lower=False)
tr4w.get_keywords(10)

system - 3.9298789569107666
PS4 - 3.2873652208751394
gaming - 2.1534122414704107
PlayStation - 1.777032178296869
entertainment - 1.6504670134534094
experiences - 1.5937993202166947
games - 1.5757554934271645
journey - 1.346523341839697
worlds - 1.2433484842414528
gamers - 1.2261288426700112
settings - 1.1900458333333335
life - 1.1900458333333335


In [0]:
#download pretrained word embedding model and load it
##download a file
##unzip a file
def download_and_unzip_pretrained_word_embedding(url_to_word_embedding_zip: str, download_destination_folder_with_slash_at_end: str) -> Tuple[bool, str]:
  #download if unzipped file exists already
  #if not, unzip and check if file exists after unzipping
  #if file exists, return 1 for success
  import os.path
  from os import path
  zipped_file_name = os.path.basename(url_to_word_embedding_zip)
  unzipped_file_name = os.path.splitext(zipped_file_name)[0]

  path_of_zipped_word_embedding = download_destination_folder_with_slash_at_end + zipped_file_name
  desired_path_of_unzipped_word_embedding = download_destination_folder_with_slash_at_end + unzipped_file_name
  zip_file_extension = os.path.splitext(url_to_word_embedding_zip)[1][1:]

  if path.exists(desired_path_of_unzipped_word_embedding) == False:
    !wget -c $url_to_word_embedding_zip -P $download_destination_folder_with_slash_at_end
    if zip_file_extension == 'gz':
      !gunzip $path_of_zipped_word_embedding
    elif zip_file_extension == 'zip':
      print("zip_file_extension", " not implemented yet")
      return (False, None)
    else:
      print("This function doesnt handle zipped files with extension: ", zip_file_extension)
      return (False, None)
  else:
    print("datasetAlreadyExists")

  if path.exists(desired_path_of_unzipped_word_embedding) == True:
    successful_in_creating_word_embedding = True
  else:
    successful_in_creating_word_embedding = False
  return (successful_in_creating_word_embedding, desired_path_of_unzipped_word_embedding)

##load word embedding
def load_word_embedding_into_KeyedVectors_model(word_embedding_file_path: str) -> gensim.models.keyedvectors.Word2VecKeyedVectors:
  from gensim.models.keyedvectors import KeyedVectors
  model = KeyedVectors.load_word2vec_format(word_embedding_file_path, binary=True)
  return model

#load dataset
#extract desired features from dataset
def load_dataset_into_pandas(url_of_dataset: str, tuple_of_starting_and_ending_exclusive_index_of_sample: tuple) -> pd.core.frame.DataFrame:
  #Create pandas dataframe on dataset
  import pandas as pd
  #https://data.world/crowdflower/ecommerce-search-relevance
  df = pd.read_csv(url_of_dataset, encoding = "ISO-8859-1")
  start,end = tuple_of_starting_and_ending_exclusive_index_of_sample
  df = df.iloc[start:end]
  return df


#preprocess data
class PreprocessData:
  """
  A class used to find the orientation of the robot with respect to a lane over time.
  """
  def __init__(self):
    """
    Create a PreprocessData object
    Parameters:
      - 
    """
    from nltk import download
    from nltk import word_tokenize
    # Import and download stopwords from NLTK.
    from nltk.corpus import stopwords
    
    download('punkt')  # Download data for tokenizer.
    download('stopwords')  # Download stopwords list.

    # Remove stopwords.
    self.stop_words = stopwords.words('english')

  def make_text_lower_case(self, text: str) -> str:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    return text.lower()

  def tokenize_string_to_list_of_separate_words(self, text_string: str) -> List[str]:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    from nltk import word_tokenize
    return word_tokenize(text_string) 

  def remove_stopwords_from_text(self, text_list: List[str]) -> List[str]:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    return [w for w in text_list if not w in self.stop_words]

  def remove_numbers_and_punctuation_from_text(self, text_list: List[str]) -> List[str]:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    return [w for w in text_list if w.isalpha()]

  def text_preprocessing_for_WMD(self, text: str) -> List[str]:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    text = self.make_text_lower_case(text)
    text_list = self.tokenize_string_to_list_of_separate_words(text)
    text_list = self.remove_stopwords_from_text(text_list)
    text_list = self.remove_numbers_and_punctuation_from_text(text_list)
    return text_list

  def text_preprocessing_with_textrank_for_WMD(self, text: str) -> List[str]:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    text = self.make_text_lower_case(text)
    
    tr4w = TextRank4Keyword()
    tr4w.analyze(text, candidate_pos = ['NOUN', 'PROPN', 'VERB'], window_size=4, lower=False)
    text_list = tr4w.get_list_of_strings_of_the_top_textrank_keywords_in_order_of_textrank(10)

    # text_list = self.tokenize_string_to_list_of_separate_words(text)
    text_list = self.remove_stopwords_from_text(text_list)
    text_list = self.remove_numbers_and_punctuation_from_text(text_list)
    return text_list

  def create_new_field_with_preprocessed_text_in_dataframe(self, dataframe: pd.core.frame.DataFrame, field_to_preprocess: str, new_field_name: str) -> pd.core.frame.DataFrame:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    num_rows_dataframe = len(dataframe.index)
    preprocessed_text_list = [np.nan] * num_rows_dataframe
    empty_text = []
    for ind in dataframe.index:
      field_text = dataframe[field_to_preprocess][ind]
      #check if field is empty or nan
      if field_text == field_text:
        preprocessed_text = self.text_preprocessing_for_WMD(field_text)
        # preprocessed_text = self.text_preprocessing_with_textrank_for_WMD(field_text)
        if preprocessed_text != []:
          preprocessed_text_list[ind] = preprocessed_text
        else:
          preprocessed_text_list[ind] = empty_text
      else:
        preprocessed_text_list[ind] = empty_text

    dataframe.insert(len(dataframe.columns), new_field_name, preprocessed_text_list, True)
    return dataframe
    
  def create_new_field_with_preprocessed_text_from_multiple_fields_in_dataframe(self, dataframe: pd.core.frame.DataFrame, fields_to_preprocess_in_order_of_appending: List[str], new_field_name: str) -> pd.core.frame.DataFrame:
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    num_rows_dataframe = len(dataframe.index)
    preprocessed_text_list = [np.nan] * num_rows_dataframe
    empty_text = []

    for ind in dataframe.index:
      list_of_field_text = []
      for i in range(len(fields_to_preprocess_in_order_of_appending)):
        if dataframe[fields_to_preprocess_in_order_of_appending[i]][ind] == dataframe[fields_to_preprocess_in_order_of_appending[i]][ind]:
          list_of_field_text.append(dataframe[fields_to_preprocess_in_order_of_appending[i]][ind])
      field_text = " ".join(filter(None, list_of_field_text))
      #check if field is empty or nan
      if field_text == field_text:
        preprocessed_text = self.text_preprocessing_for_WMD(field_text)
        # preprocessed_text = self.text_preprocessing_with_textrank_for_WMD(field_text)
        if preprocessed_text != []:
          preprocessed_text_list[ind] = preprocessed_text
        else:
          preprocessed_text_list[ind] = empty_text
      else:
        preprocessed_text_list[ind] = empty_text

    dataframe.insert(len(dataframe.columns), new_field_name, preprocessed_text_list, True)
    return dataframe     

#find WMD similarity
class WordMoversDistanceSimilarity:
  """
  A class used to find the orientation of the robot with respect to a lane over time.
  """
  def __init__(self, keyed_vector_model: gensim.models.keyedvectors.Word2VecKeyedVectors, list_of_document_lists: List[List[str]], num_similar_queries: int):
    """
    Create a PreprocessData object
    Parameters:
      - 
    """
    self.model = keyed_vector_model
    self.wmd_corpus = list_of_document_lists
    self.num_similar_queries = num_similar_queries
    self.preprocess_data_object = PreprocessData()
    self.similarityMatrix = 0
    self.similar_document_tuple_of_indexes_and_similarity_scores = 0

  def create_similarity_matrix(self):
    """
    make text lower case. Returns None if text is empty
    Params:
      - text: string of the input text we want to make lowercase
    Returns:
      - lower_case_text: the input text made lowercase
    """
    from gensim.similarities import WmdSimilarity
    self.similarityMatrix = WmdSimilarity(self.wmd_corpus, self.model, self.num_similar_queries)

  def find_similar_document_index_and_similarity(self, query_text: str) -> List[tuple]:
    query_text = preprocess(source_doc)
    self.similar_document_tuple_of_indexes_and_similarity_scores = instance[query]  # A query is simply a "look-up" in the similarity class.
    return self.similar_document_tuple_of_indexes_and_similarity_scores

class PrintingResultAndCalculateLoss:
  """
  A class used to find the orientation of the robot with respect to a lane over time.
  """
  def __init__(self, similar_document_tuple_of_indexes_and_similarity_scores: int, data_frame: pd.core.frame.DataFrame):
    """
    Create a PreprocessData object
    Parameters:
      - 
    """
    self.similar_document_tuple_of_indexes_and_similarity_scores = similar_document_tuple_of_indexes_and_similarity_scores
    self.data_frame = data_frame

  def calculate_accuracy_of_similarity_prediction_for_a_query_text_from_actual_(index_of_query_product, score_cutoff):
    """
    Calculate accuracy for a given single similar products query. This query text must be the actual data from the table, so that it will have a 
    label "query" field and "relevance" field.

    if index_of_query_product "relevance" field < score_cutoff: #we dont want to use query items in which the similarity to the search query was low.
      return accuracy = None
    else: 
      for every index in similar_document_tuple_of_indexes_and_similarity_scores:
        number_of_correct_similar_products += (index_of_query_product "query" field == similar_index "query" field)*(similar_index "relevance" field >= score_cutoff)
    accuracy = number_of_correct_similar_products/num_similar_queries

    Parameters:
      - 
    """
  #in order to be classified correctly, the similar product has to be in same search query as target category
  #and has to have a relavance score >= score cutoff.
  #I know this is flawed because there might be a product in the wrong search query but is still similar to the target product.
  #but overall, im hoping this case to be less

  #
    num_similar_queries = len(index_of_query_product)

    search_query = df['query'][sims[i][0]]
    for i in range(num_similar_queries):
      index_of_

def calculate_output_dataframe_accuracy_and_write_in_dataframe(output_dataframe, relevance_threshold):
  numRows = output_dataframe["similarDescription"].shape[0]
  numMatchingCategory = 0
  numNonMatchingCategory = 0
  if output_dataframe["targetRelevanceToQuery"][0] < relevance_threshold:
    accuracy, numMatchingCategory, numNonMatchingCategory = (-1,-1,-1)
    return accuracy, numMatchingCategory, numNonMatchingCategory
  for i in range(numRows):  
    if output_dataframe["targetSearchQuery"][0] == output_dataframe["similarSearchQuery"][i]:
      numMatchingCategory += 1
    else:
      numNonMatchingCategory +=1
  print("numMatchingCategory = ", numMatchingCategory)
  print("numNonMatchingCategory = ", numNonMatchingCategory)
  ##ipdb.set_trace()
  accuracy = numMatchingCategory/numRows
  output_dataframe["WMDAccuracy"][0] = accuracy
  return accuracy, numMatchingCategory, numNonMatchingCategory


In [31]:

############################
##download model and load model
##Inputs
url_to_word_embedding_zip = "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
download_destination_folder_with_slash_at_end = "data/"

download_success, desired_path_of_unzipped_word_embedding = download_and_unzip_pretrained_word_embedding(url_to_word_embedding_zip, download_destination_folder_with_slash_at_end)
word_embedding_file_path = desired_path_of_unzipped_word_embedding
model = load_word_embedding_into_KeyedVectors_model(word_embedding_file_path)



datasetAlreadyExists


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


In [45]:

#setup pretrained model

# unitIdToGetIndexesOf = [711158459, 711158484]

############################
#setup dataset
##Inputs
url_of_dataset = 'https://query.data.world/s/iccclhgzbyedtv54p5lresglnun3qz'
tuple_of_starting_and_ending_exclusive_index_of_sample = (0,500)

dataset_frame = load_dataset_into_pandas(url_of_dataset, tuple_of_starting_and_ending_exclusive_index_of_sample)
list_of_queries_field = dataset_frame['query'].unique().tolist()

indexesOfTargetItems = []
for i in range(len(list_of_queries_field)):
  # unitId = list_of_queries_field[i]
  # indexesOfTargetItems.append(pd.Index(dataset_frame['_unit_id']).get_loc(unitId))
  # print(i)
  # print(pd.Index(dataset_frame['query']).get_loc(list_of_queries_field[i]))
  if isinstance(pd.Index(dataset_frame['query']).get_loc(list_of_queries_field[i]), int):
    indexesOfTargetItems.append(pd.Index(dataset_frame['query']).get_loc(list_of_queries_field[i]))
  else:
    indexesOfTargetItems.append(pd.Index(dataset_frame['query']).get_loc(list_of_queries_field[i]).argmax())
print(list_of_queries_field)
# ipdb.set_trace()

['playstation 4', 'eye cream', 'routers', 'shaver panasonic', 'coffee grinder', 'high chairs', 'face cream', 'Brett Favre NY Titans jersey blue', 'aroma diffuser', 'Bluray Hobbit extended', 'white plain dinner set', 'double stroller', 'Vanilla Scented Perfumes', 'wireless mouse', 'spiderman', 'bedspreads', 'kitchen rugs', 'LED monitor', 'PS2 controller USB', 'extenal hardisk 500 gb', 'laptop lenovo']


In [46]:

############################
for source_doc_index in indexesOfTargetItems:
  url_of_dataset = 'https://query.data.world/s/iccclhgzbyedtv54p5lresglnun3qz'
  tuple_of_starting_and_ending_exclusive_index_of_sample = (0,500)

  dataset_frame = load_dataset_into_pandas(url_of_dataset, tuple_of_starting_and_ending_exclusive_index_of_sample)

  ############################
  ##preprocess text
  ##create a new field that has
  ##Inputs
  # dataframe = dataset_frame
  # field_to_preprocess = 'product_description'
  # new_field_name = 'product_description_preprocessed'

  # preprocess_data_object = PreprocessData()
  # #create_new_field_with_preprocessed_text_in_dataframe(self, dataframe: pd.core.frame.DataFrame, field_to_preprocess: str, new_field_name: str) -> pd.core.frame.DataFrame:
  # dataframe_modified = preprocess_data_object.create_new_field_with_preprocessed_text_in_dataframe(dataframe, field_to_preprocess, new_field_name)

  ############################
  #alternate preprocess text
  ##Inputs
  dataframe = dataset_frame
  # fields_to_preprocess_in_order_of_appending = ['product_title', 'product_description']
  fields_to_preprocess_in_order_of_appending = ['product_title']
  new_field_name = ",".join(fields_to_preprocess_in_order_of_appending)+'_preprocessed'

  print("new_field_name = ", new_field_name)

  preprocess_data_object = PreprocessData()
  #create_new_field_with_preprocessed_text_in_dataframe(self, dataframe: pd.core.frame.DataFrame, field_to_preprocess: str, new_field_name: str) -> pd.core.frame.DataFrame:
  dataframe_modified = preprocess_data_object.create_new_field_with_preprocessed_text_from_multiple_fields_in_dataframe(dataframe, fields_to_preprocess_in_order_of_appending, new_field_name)

  ############################
  #Create Word movers distance similarity matrix
  #Inputs
  num_best_var = 20
  # source_doc_index = 0
  print("source_doc_index = ", source_doc_index)

  source_doc_preprocessed = dataframe_modified[new_field_name][source_doc_index]
  print(source_doc_preprocessed)

  from gensim.similarities import WmdSimilarity
  wmd_corpus = dataframe_modified[new_field_name].tolist()
  wmd_object_for_description = WmdSimilarity(wmd_corpus, model, num_best = num_best_var)

  similar_item_indexes_and_similarity = wmd_object_for_description[source_doc_preprocessed]

  similar_product_index = []
  similar_product_similarities =[]

  for i in range(num_best_var):
    print(i)
    similar_product_index.append(similar_item_indexes_and_similarity[i][0])
    similar_product_similarities.append(similar_item_indexes_and_similarity[i][1])

  target_index = ['']*num_best_var
  target_title_list = ['']*num_best_var
  target_search_query_list = ['']*num_best_var
  target_search_relevance_to_query = ['']*num_best_var
  target_description_list = ['']*num_best_var
  target_description_preprocesed_list = ['']*num_best_var
  wmd_accuracy = ['']*num_best_var

  target_index[0] = source_doc_index
  target_title_list[0] = dataframe_modified['product_title'][source_doc_index]
  target_search_query_list[0] = dataframe_modified['query'][source_doc_index]
  target_search_relevance_to_query[0] = dataframe_modified['relevance'][source_doc_index]
  target_description_list[0] = dataframe_modified['product_description'][source_doc_index]
  target_description_preprocesed_list[0] = dataframe_modified[new_field_name][source_doc_index]

  data = {'targetIndex': target_index, 
          'targetTitle': target_title_list, 
          'targetSearchQuery': target_search_query_list,
          'targetRelevanceToQuery': target_search_relevance_to_query, 
          'targetDescription': target_description_list,
          'target'+new_field_name: target_description_preprocesed_list,
          'similarIndex': similar_product_index,
          'similarTitle': dataframe_modified['product_title'][similar_product_index].tolist(),
          'similarSearchQuery': dataframe_modified['query'][similar_product_index].tolist(),
          'similarRelevanceToQuery': dataframe_modified['relevance'][similar_product_index].tolist(),
          'similarDescription': dataframe_modified['product_description'][similar_product_index].tolist(),
          'similar'+new_field_name: dataframe_modified[new_field_name][similar_product_index].tolist(),
          'WMDRank': list(range(0, num_best_var)),
          'WMDSimilarity': similar_product_similarities, 
          'WMDAccuracy': wmd_accuracy} 
  output_dataframe = pd.DataFrame(data)
  relevance_threshold = 2
  accuracy, numMatchingCategory, numNonMatchingCategory = calculate_output_dataframe_accuracy_and_write_in_dataframe(output_dataframe, relevance_threshold)
  output_csv_name = 'csv_logs_only_title_06_14_20/searchQuery_'+dataframe_modified['query'][source_doc_index]+'_numMatches_'+str(num_best_var)

  output_dataframe.to_csv(output_csv_name+'.csv')



# wmd_corpus_for_description = create_corpus_from_preprocessed_text_field(self, dataframe: pd.core.frame.DataFrame, ) -> List[List[str]]
#   dataframe['preprocessed_description'].toList()
# WMD_object_for_description = create_WMD_object(wmd_corpus, model, num_best)
# similar_item_indexes_and_similarity = get_similar_item_indexes_and_similarity()

# print_out_similar_item_details_to_dataframe


# #Find the similarities for one item in the search query, and display the fields of title, relevance, description, wmd ranking, and wmd rating
# download_and_unzip_pretrained_word_embedding

new_field_name =  product_title_preprocessed
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
source_doc_index =  0
['sony', 'playstation', 'latest', 'model', 'gb', 'jet', 'black', 'console']
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
numMatchingCategory =  20
numNonMatchingCategory =  0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:299: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


new_field_name =  product_title_preprocessed
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
source_doc_index =  25
['new', 'clinique', 'repairwear', 'laser', 'focus', 'wrinkle', 'correcting', 'eye', 'cream']
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
numMatchingCategory =  17
numNonMatchingCategory =  3
new_field_name =  product_title_preprocessed
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
source_doc_index =  50
['router']
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
numMatchingCategory =  20
numNonMatchingCategory =  0
new_field_name =  product_title_preprocessed
[nltk_data] Downloading package punkt to /root/

In [9]:
import numpy as np
import pandas as pd
import glob
import ipdb
def calculate_average_accuracy_from_csv_logs(folder_path_of_csv_logs: str) ->  Tuple[float, pd.core.frame.DataFrame]:
  #get paths of all the csv files in the folder_path_of_csv_logs directory
  all_files = glob.glob(folder_path_of_csv_logs + "*.csv")

  #initialize lists to feed into output dataframe later
  search_query_list = []
  accuracy_list = []

  #for each file:
  for filename in all_files:
    #load the file into pandas
    df = pd.read_csv(filename)
     #read the accuracy value, and save it into a pandas dataframe with fields: search query, accuracy
    accuracy_list.append(df['WMDAccuracy'][0])
    search_query_list.append(df['targetSearchQuery'][0])
  
  #calculate average of all the accuracies
  average_accuracy = sum(accuracy_list)/len(accuracy_list)
   
  #return dictionary of dataframe, as well as average accuracy
  search_query_accuracy_dataframe = {
      'searchQuery': search_query_list, 
      'accuracy': accuracy_list, 
      } 
  output_dataframe = pd.DataFrame(search_query_accuracy_dataframe)

  return average_accuracy, output_dataframe
#####################
folder_path_of_csv_logs = 'csv_logs_only_title_06_14_20/'
# ipdb.set_trace()
average_accuracy, output_dataframe = calculate_average_accuracy_from_csv_logs(folder_path_of_csv_logs)
print("average accuracy of ", folder_path_of_csv_logs, " = ", average_accuracy)
output_dataframe

average accuracy of  csv_logs_only_title_06_14_20/  =  0.8904761904761904


,searchQuery,accuracy
0,playstation 4,1.00
1,eye cream,0.85
2,routers,1.00
3,shaver panasonic,1.00
4,coffee grinder,1.00
5,high chairs,0.25
6,face cream,0.60
7,Brett Favre NY Titans jersey blue,1.00
8,aroma diffuser,1.00
9,Bluray Hobbit extended,1.00


In [0]:
import unittest
class MyTest(unittest.TestCase):
  @unittest.skip("demonstrating skipping")
  def test_download_and_unzip_pretrained_word_embedding(self):
    url_to_word_embedding_zip = "http://dumps.wikimedia.org/other/articlefeedback/aa_combined-20110321.csv.gz"
    download_destination_path = "data/"
    path_to_word_embedding  = download_destination_path + "aa_combined-20110321.csv"
    #ipdb.set_trace()
    !rm $path_to_word_embedding
    success, desired_path_of_unzipped_word_embedding = download_and_unzip_pretrained_word_embedding(url_to_word_embedding_zip, download_destination_path)
    self.assertEqual(success, 1)

  @unittest.skip("demonstrating skipping")
  def test_load_word_embedding_into_KeyedVectors_model(self):
    url_to_word_embedding_zip = "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
    download_destination_folder_with_slash_at_end = "data/"

    download_success, desired_path_of_unzipped_word_embedding = download_and_unzip_pretrained_word_embedding(url_to_word_embedding_zip, download_destination_folder_with_slash_at_end)
    if download_success == 0:
      success = 0
    else:
      word_embedding_file_path = desired_path_of_unzipped_word_embedding
      model = load_word_embedding_into_KeyedVectors_model(word_embedding_file_path)
      
      if isinstance(model, gensim.models.keyedvectors.Word2VecKeyedVectors):
        success =1
      else:
        success = 0
    
    self.assertEqual(success, 1)

  @unittest.skip("demonstrating skipping")
  def test_load_dataset_into_pandas(self):
    url_of_dataset = 'https://query.data.world/s/iccclhgzbyedtv54p5lresglnun3qz'
    tuple_of_starting_and_ending_exclusive_index_of_sample = (0,10)
    dataset_frame = load_dataset_into_pandas(url_of_dataset, tuple_of_starting_and_ending_exclusive_index_of_sample)
    
    #pd.set_option('display.max_columns', None)
    #print(dataset_frame)
    
    self.assertEqual(isinstance(dataset_frame, pd.core.frame.DataFrame),1) 
    self.assertEqual(dataset_frame.shape[0] == (tuple_of_starting_and_ending_exclusive_index_of_sample[1]-tuple_of_starting_and_ending_exclusive_index_of_sample[0]), 1)
  
  def test_create_new_field_with_preprocessed_text_in_dataframe(self):
    url_of_dataset = 'https://query.data.world/s/iccclhgzbyedtv54p5lresglnun3qz'
    tuple_of_starting_and_ending_exclusive_index_of_sample = (0,10)
    dataset_frame = load_dataset_into_pandas(url_of_dataset, tuple_of_starting_and_ending_exclusive_index_of_sample)
    
    dataframe = dataset_frame
    field_to_preprocess = 'product_description'
    new_field_name = 'product_description_preprocessed'
    
    preprocess_data_object = PreprocessData()
    #create_new_field_with_preprocessed_text_in_dataframe(self, dataframe: pd.core.frame.DataFrame, field_to_preprocess: str, new_field_name: str) -> pd.core.frame.DataFrame:
    dataframe_modified = preprocess_data_object.create_new_field_with_preprocessed_text_in_dataframe(dataframe, field_to_preprocess, new_field_name)
    #end of function
    #self.assertEqual(isinstance(dataframe_modified, pd.core.frame.DataFrame),1) 
    pd.set_option('display.max_columns', None)
    print(dataframe_modified)
    

if __name__ == '__main__':
  unittest.main(argv=['first-arg-is-ignored'], exit=False)


.sss

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
    _unit_id  relevance  relevance:variance  \
0  711158459       3.67               0.471   
1  711158460       4.00               0.000   
2  711158461       4.00               0.000   
3  711158462       3.67               0.471   
4  711158463       3.33               0.471   
5  711158464       3.20               0.748   
6  711158465       4.00               0.000   
7  711158466       4.00               0.000   
8  711158467       3.75               0.433   
9  711158468       2.33               0.471   

                                       product_image  \
0  http://thumbs2.ebaystatic.com/d/l225/m/mzvzEUI...   
1  http://thumbs3.ebaystatic.com/d/l225/m/mJNDmSy...   
2  http://thumbs4.ebaystatic.com/d/l225/m/m10NZXA...   
3  http://thumbs2.ebaystatic


----------------------------------------------------------------------
Ran 4 tests in 2.871s

OK (skipped=3)


In [0]:
dataframe_modified['product_title'][0]

'Sony PlayStation 4 (PS4) (Latest Model)- 500 GB Jet Black Console'

In [0]:
print(output_dataframe["similarDescription"].shape)
numMatchingCategory

(20,)


11

In [0]:
#INPUTS

#repoPath should start with the path 
#'/content/gdrive/My Drive/' and go to the folder where the git repo is, and should end with the name of the repo:
repoPath = '/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance'
userEmail = 'sanmesh1@gmail.com'
userName = 'sanmesh1'

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [13]:
! pip install ipdb -q
import ipdb 

#place the line of code below from whichever line you want to start debugging
#type n for step over and s for step into
# ipdb.set_trace()

In [0]:
import os
os.chdir(repoPath)
#%cd gdrive/My Drive/project_folder/TextSimilarityUsingWord2Vec

In [0]:
from time import time
start_nb = time()

In [0]:
# Import and download stopwords from NLTK.
from nltk.corpus import stopwords
from nltk import download
download('stopwords')  # Download stopwords list.

# Remove stopwords.
stop_words = stopwords.words('english')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [0]:
start = time()
import os
from gensim.models import Word2Vec

%cd data/
!ls
import os.path
from os import path
if path.exists("GoogleNews-vectors-negative300.bin") == False:
  !wget -c "https://s3.amazonaws.com/dl4j-distribution/GoogleNews-vectors-negative300.bin.gz"
  !gunzip GoogleNews-vectors-negative300.bin
else:
  print("datasetAlreadyExists")
%cd ..

from gensim.models.keyedvectors import KeyedVectors
model_path = './data/GoogleNews-vectors-negative300.bin'
model = KeyedVectors.load_word2vec_format(model_path, binary=True)

print('Cell took %.2f seconds to run.' % (time() - start))

/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance/data
GoogleNews-vectors-negative300.bin
datasetAlreadyExists
/content/gdrive/My Drive/TestingGitCloneIntoGoogleDrive/textSimilarityUsingWordMoversDistance


/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:253: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


Cell took 83.21 seconds to run.


In [0]:
# Pre-processing a document.

from nltk import word_tokenize
download('punkt')  # Download data for tokenizer.

def preprocess(doc):
    doc = doc.lower()  # Lower the text.
    doc = word_tokenize(doc)  # Split into words.
    doc = [w for w in doc if not w in stop_words]  # Remove stopwords.
    doc = [w for w in doc if w.isalpha()]  # Remove numbers and punctuation.
    return doc

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
#Create pandas dataframe on dataset
import pandas as pd
#https://data.world/crowdflower/ecommerce-search-relevance
df = pd.read_csv('https://query.data.world/s/iccclhgzbyedtv54p5lresglnun3qz', encoding = "ISO-8859-1")

In [0]:
#create source document and target document we will be comparing scores for
numDataPoints = 6000

sourceTitle = df['product_title'][0]
source_doc = df['product_description'][0] #2584

targetTitles = df['product_title'][:numDataPoints].tolist()
target_docs = df['product_description'][:numDataPoints].tolist()
targetQueries = df['query'][:numDataPoints].tolist()

In [0]:
#using WmdSimilarity

wmd_corpus = []
documents = []
documentTitles = []
documentQueries = []

for i in range(len(target_docs)):
    if target_docs[i] == target_docs[i]: #this is to check if there is an empty string
      # ipdb.set_trace()
      target_docs_preproc = preprocess(target_docs[i])
      if target_docs_preproc != []:
        wmd_corpus.append(target_docs_preproc)
      else:
        wmd_corpus.append([])
    else:
      wmd_corpus.append([])
    documentTitles.append(targetTitles[i])
    documents.append(target_docs[i])
    documentQueries.append(targetQueries[i])
# Initialize WmdSimilarity.
from gensim.similarities import WmdSimilarity
num_best = 10
instance = WmdSimilarity(wmd_corpus, model, num_best=10)



In [0]:
start = time()

query = preprocess(source_doc)

sims = instance[query]  # A query is simply a "look-up" in the similarity class.
print('Cell took %.2f seconds to run.' % (time() - start))

Cell took 415.79 seconds to run.


In [0]:
# Print the query and the retrieved documents, together with their similarities.
print('Query:')
print(sourceTitle)
for i in range(num_best):
    print()
    print('sim in description = %.4f' % sims[i][1])
    print(documentTitles[sims[i][0]])
    simInTitles = model.wmdistance(sourceTitle, documentTitles[sims[i][0]])
    print("loss in title similarity = ", simInTitles)
    print("item query = ", documentQueries[sims[i][0]])

Query:
Sony PlayStation 4 (PS4) (Latest Model)- 500 GB Jet Black Console

sim in description = 1.0000
Sony PlayStation 4 500GB, Dualshock Wireless Control, HDMI Gaming Console Refurb
loss in title similarity =  0.3408880498625308
item query =  playstation 4

sim in description = 1.0000
Sony PlayStation 4 (Latest Model)- 500 GB Jet Black Console *NEW*
loss in title similarity =  0.10508894400103874
item query =  playstation 4

sim in description = 1.0000
Sony PlayStation 4 500GB Console with 2 Controllers
loss in title similarity =  0.2689347376582511
item query =  playstation 4

sim in description = 1.0000
Sony PlayStation 4 (Latest Model) 500 GB Jet Black Console
loss in title similarity =  0.06799642897847154
item query =  playstation 4

sim in description = 1.0000
Sony PlayStation 4 (PS4) (Latest Model)- 500 GB Jet Black Console
loss in title similarity =  0.0
item query =  playstation 4

sim in description = 1.0000
Sony PlayStation 4 PS4 500 GB Jet Black Console
loss in title simil

In [0]:
def calculatePercentageOfSimilarItemsInSameQueryCategory(instance, df, num_best, numSamples):
  numRows = df.shape[0]
  numMatchingCategory = 0
  numNonMatchingCategory = 0
  # for i in range(numRows):  
  for i in range(numSamples):  
    print(i)
    source_doc_query = df['query'][i]
    if source_doc_query == source_doc_query:
      query = preprocess(source_doc_query)
      sims = instance[query]  # A query is simply a "look-up" in the similarity class.
      for i in range(num_best):
        if documentQueries[sims[i][0]] == source_doc_query:
          numMatchingCategory += 1
        else:
          numNonMatchingCategory
  print("numMatchingCategory = ", numMatchingCategory)
  print("numNonMatchingCategory = ", numNonMatchingCategory)
  return numMatchingCategory, numNonMatchingCategory



In [0]:
numMatchingCategory, numNonMatchingCategory =calculatePercentageOfSimilarItemsInSameQueryCategory(instance, df, num_best, 10)

0
1
2
3
4
5
6
7
8
9
numMatchingCategory =  20
numNonMatchingCategory =  0
